In [1]:
import h5py
#import pylab as pl
import numpy as np
#import matplotlib as mpl
import networkx
import ipyvolume as ipv

In [2]:
def split_unique_id(unique_id):
    """Splits the ids assign to the subhalos by the merger tree code by snap number and subfind number """
    subfind_number = int(unique_id % 1e6)
    snap_number = int((unique_id - subfind_number) / 1e6)
    
    return snap_number, subfind_number

def get_main_branch_unique_ids(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    i = 0
    while True:
        succesors = list(subtree.successors(node))
        if len(succesors) == 0:
            break
        node = succesors[0] # select only the first succesor (main branch)
        mpb.append(node)
        
    return mpb

def get_main_branch_and_progNumb(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    numb = [[node], ]
    i = 0
    while True:
        successors = list(subtree.successors(node))
        if len(successors) == 0:
            break
        node = successors[0] # select only the first succesor (main branch)
        mpb.append(node)
        numb.append(successors)
        
    return mpb, numb


def set_view(figure, framenr, fraction):
    ipv.view(fraction*360, 0.)
    
def set_angles(fig, i, fraction):
    fig.angley = fraction*np.pi*2

In [3]:
sim = h5py.File('/data/cielo/simulations/LG1/LG1.hdf5', 'r')
trees = networkx.read_multiline_adjlist('/data/cielo/simulations/LG1/LG1_tree.dat')

In [4]:
def plotItIn3D(fofID, centralID):
    uniqueID = int(127*1e6+fofID)
    stree = networkx.dfs_tree(trees, str(uniqueID))
    mtree, numbs = get_main_branch_and_progNumb(stree, str(uniqueID))
    
    uniqueC = int(127*1e6+centralID)
    streeC = networkx.dfs_tree(trees, str(uniqueC))
    mtreeC = get_main_branch_unique_ids(streeC, str(uniqueC))
    
    # find all the particleIDs that once belong to the subgroup
    Idlist, zlist, zcen = [], [], []

    galH, cenH = [], []
    
    for mm, mmC in zip(mtree[1:], mtreeC[1:]): # ignore the first one
        snap, idd = split_unique_id(int(mm))
        snapC, iddC = split_unique_id(int(mmC))
        tempOff = sim['SnapNumber_{}/SubGroups/PartType0/Offsets'.format(snap)][idd].astype('int')
        pgal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][idd]
        zt = sim['SnapNumber_{}/Header/Redshift'.format(snap)][()]
        if snapC==snap:
            pcen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snapC)][iddC]
        galH.append(pgal)
        cenH.append(pcen)
        zcen.append(zt)
        if tempOff[0]>= 0 and tempOff[1]>=0:
            tempIds = sim['SnapNumber_{}/PartType0/ParticleIDs'.format(snap)][tempOff[0]:tempOff[1]]
            Idlist.extend(list(tempIds))
            zlist.extend([zt]*tempIds.size)

    Idlist, zlist = np.array(Idlist), np.array(zlist)
    Idlist, Idunique = np.unique(Idlist, return_index=True) # remove duplicates
    zlist = zlist[Idunique]

    galH = np.array(galH)
    cenH = np.array(cenH)
    zcen = np.array(zcen)
    
    # the total list of particles that once belong to a give subgroup, lest see where are today.
    TdayOff = sim['SnapNumber_127/SubGroups/PartType0/Offsets'][fofID].astype('int')
    TdayCoord = sim['SnapNumber_127/PartType0/Coordinates'][TdayOff[0]:TdayOff[1]]

    TdayTotalIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][()]
    
    IndexToday = np.in1d(TdayTotalIDs, Idlist)
    IndexToday_ = np.where(IndexToday)[0]
    
    #remove the ones that are not part of the group today.
    TdayIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][TdayOff[0]:TdayOff[1]]

    IndexNot = np.in1d(Idlist, TdayIDs)
    IdNot = Idlist[np.where(IndexNot==False)[0]]

    IndexT = np.in1d(TdayTotalIDs, IdNot)
    IndexT_ = np.where(IndexT)[0]
    LostCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_]

    # ... and the redshift
    SurvIds_ = TdayTotalIDs[IndexT_]
    iidex = np.in1d(Idlist, SurvIds_)
    iidex_ = np.where(iidex)[0]
    zlist_, Idlist_ = zlist[iidex_], Idlist[iidex_]

    is1 = np.argsort(SurvIds_) # the target
    is2 = np.argsort(Idlist_)

    zorder = np.zeros(IndexT_.size)
    zorder[is1] = zlist_[is2]
    
    #now lets cut at  a certain distance
    distance = 600 # distance in ckpc

    posGal = sim['SnapNumber_127/SubGroups/SubGroupPos'][fofID]
    posCen = sim['SnapNumber_127/SubGroups/SubGroupPos'][centralID]

    dx, dy, dz = LostCoord[:,0]-posGal[0], LostCoord[:,1]-posGal[1], LostCoord[:,2]-posGal[2]
    distToC = np.sqrt(dx**2+dy**2+dz**2)

    icut = np.where(distToC<distance)[0]
    #IndexT_near = IndexT_[icut]
    #zorder_near = zorder[icut]
    IndexT_near = IndexT_
    zorder_near = zorder
    
    Index_limited = IndexT_[icut]
    
    NearCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_near]
    NearestCoord = sim['SnapNumber_127/PartType0/Coordinates'][Index_limited]
    
    # ok, let's see velocity and stuff
    velCen = sim['SnapNumber_128/SubGroups/SubGroupVel'][centralID]
    velSat = sim['SnapNumber_128/SubGroups/SubGroupVel'][fofID]

    cenRep = posGal - posCen

    velRep = velSat - velCen
    
    xo, yo, zo = NearCoord[:,0], NearCoord[:,1], NearCoord[:,2]
    xoC, yoC, zoC = xo - posCen[0], yo - posCen[1], zo - posCen[2]
    
    xoT, yoT, zoT = NearestCoord[:,0] - posCen[0], NearestCoord[:,1] - posCen[1], NearestCoord[:,2] - posCen[2]
    
    velX, velY, velZ = velRep[0], velRep[1], velRep[2]

#   cenPx, cenPy, cenPz = galH[:,0] - posCen[0], galH[:,1] - posCen[1], galH[:,2] - posCen[2]
    cenPx, cenPy, cenPz = galH[:,0] - cenH[:,0], galH[:,1] - cenH[:,1], galH[:,2] - cenH[:,2]

    nzbins = 4
    # zlim = np.linspace(zorder_near.min(), zorder_near.max(), nzbins+1)
    zli = np.percentile(zorder_near, [25, 50, 75, 100])
    zlim = [0.]
    zlim.extend(list(zli))
    zlim = np.array(zlim)

    
    for i in range(zlim.size):
        zlim[i] = float(int(zlim[i]*100))/100
    
#     zlim = [0, 0.4, 0.6, 1., zorder_near.max()]
    
    colourlist = ['blue', 'green', 'orange', 'red']
    
    fig = ipv.figure(width=750, height=750)
    
    xmin, xmax = xoT.min(), xoT.max()
    ymin, ymax = yoT.min(), yoT.max()
    zmin, zmax = zoT.min(), zoT.max()
    
    for zi, zm, colour in zip(zlim[:-1], zlim[1:], colourlist):
        print(colour,'\t:','{} < z < {}'.format(zi, zm))
        i = np.where((zorder_near>zi) & (zorder_near<zm))
        xoCT ,yoCT, zoCT = xoC[i], yoC[i], zoC[i]
        scatter = ipv.scatter(xoCT, yoCT, zoCT, marker='sphere', size=.25, color=colour)
        
        iz = np.where((zcen>zi) &(zcen<zm))
        line = ipv.plot(cenPx[iz], cenPy[iz], cenPz[iz], ls='--', color=colour)
    

#         line = ipv.plot(cenPx[iz], cenPy[iz], cenPz[iz], ls='--', color=colour)
#     scatter = ipv.scatter(xoC, yoC, zoC, marker='sphere', size=.25, color=colname)
#     line = ipv.plot(cenPx[iz], cenPy[iz], cenPz[iz], ls='--', color=colname2)
#     ipv.plot(cenPx, cenPy, cenPz, ls='--', color='black')
    quiv = ipv.quiver(np.array([cenRep[0]]), np.array([cenRep[1]]), np.array([cenRep[2]]), np.array([velX]), np.array([velY]), np.array([velZ]), color='black', size=10)
    
    #ipv.xlabel('x/(ckpc h^{-1})')
    #ipv.ylabel('y/(ckpc h^{-1})')
    #ipv.zlabel('z/(ckpc h^{-1})')
    
    for i, z in enumerate(zcen):
        if len(numbs[i]) > 1:
            main = numbs[i][0]
            snap, fofi = split_unique_id(int(main))
            offM = sim['SnapNumber_{}/SubGroups/PartType1/Offsets'.format(snap)][fofi]
            mainN = offM[1]-offM[0]
            otherN = 0
            for n in numbs[i][1:]:
                snapS, fofS = split_unique_id(int(n))
                offS = sim['SnapNumber_{}/SubGroups/PartType1/Offsets'.format(snapS)][fofS]
                otherN += offS[1] - offS[0]
            ratio = otherN/mainN
            mergers = ipv.scatter(np.array([cenPx[i]]), np.array([cenPy[i]]), np.array([cenPz[i]]), marker='sphere', size=ratio*20, color='black')
    
#     ipv.movie('{}.gif'.format(fofID), set_angles, fps=20, frames=20*4, endpoint=False)
    print(xoT.size)
    
    mmin = np.min([xmin, ymin, zmin])
    mmax = np.max([xmax, ymax, zmax])

    ipv.xlim(mmin, mmax)
    ipv.ylim(mmin, mmax)
    ipv.zlim(mmin, mmax)
    ipv.show()

In [5]:
# 33, 35, 82, 85
plotItIn3D(33,32)

blue 	: 0.0 < z < 0.62
green 	: 0.62 < z < 1.47
orange 	: 1.47 < z < 2.31
red 	: 2.31 < z < 8.45
2641


In [6]:
plotItIn3D(35,32)

blue 	: 0.0 < z < 0.11
green 	: 0.11 < z < 1.66
orange 	: 1.66 < z < 1.82
red 	: 1.82 < z < 8.89
3669


In [7]:
plotItIn3D(82,81)

blue 	: 0.0 < z < 0.22
green 	: 0.22 < z < 0.52
orange 	: 0.52 < z < 0.72
red 	: 0.72 < z < 9.86
6200


In [8]:
plotItIn3D(85,81)

blue 	: 0.0 < z < 0.91
green 	: 0.91 < z < 1.3
orange 	: 1.3 < z < 1.53
red 	: 1.53 < z < 8.02
923


In [9]:
# filename = '33.gif'
# ipv.movie(filename, set_angles, fps=20, frames=20*4, endpoint=False)

In [10]:
# 4337, 4469
allgroups = sim['/SnapNumber_128/SubGroups/GroupNumber'][()]
gr1 = sim['/SnapNumber_128/SubGroups/GroupNumber'][4337]
gr2 = sim['/SnapNumber_128/SubGroups/GroupNumber'][4469]
i1 = np.where(allgroups==gr1)[0]
i2 = np.where(allgroups==gr2)[0]


In [11]:
# lets take a look at which ones of these currently have gas particles
for i in i1:
    offset = sim['/SnapNumber_128/SubGroups/PartType0/Offsets'][i].astype('int')
    if offset[0]>=0 and offset[1]>=0:
        print(i)

4337
4338
4339
4341


In [12]:
for i in i2:
    offset = sim['/SnapNumber_128/SubGroups/PartType0/Offsets'][i].astype('int')
    if offset[0]>=0 and offset[1]>=0:
        print(i)

4469
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4480
4481
4482


In [15]:
# so I gonna use 4338, 4341, (4337) 4470, 4482 (4469), lets see.
plotItIn3D(4338,4337)

blue 	: 0.0 < z < 0.18
green 	: 0.18 < z < 0.27
orange 	: 0.27 < z < 0.48
red 	: 0.48 < z < 10.37
22297


In [16]:
#ipv.movie('4474.gif', set_angles, fps=20, frames=20*4, endpoint=False)

In [17]:
plotItIn3D(4341,4337)

blue 	: 0.0 < z < 0.21
green 	: 0.21 < z < 0.27
orange 	: 0.27 < z < 0.99
red 	: 0.99 < z < 10.37
7136


In [18]:
plotItIn3D(4470,4469)

blue 	: 0.0 < z < 0.44
green 	: 0.44 < z < 0.54
orange 	: 0.54 < z < 1.03
red 	: 1.03 < z < 8.89
42105


In [19]:
plotItIn3D(4474,4469)

blue 	: 0.0 < z < 0.24
green 	: 0.24 < z < 0.34
orange 	: 0.34 < z < 0.78
red 	: 0.78 < z < 8.45
11203
